In [1]:
import sys
sys.path.append("../..")
from common import *
from ad9174 import Ad9174Settings, Ad9174Init

In [2]:
Ad9174Settings(json_file='../build/csr.json')

jesd_import ADJCNT 0
jesd_import ADJDIR 0
jesd_import BID 5
jesd_import CF 0
jesd_import CS 0
jesd_import DID 90
jesd_import DSP_CLK_DIV 32
jesd_import F 4
jesd_import FCHK 230
jesd_import FCHK_OVER_OCTETS True
jesd_import FR_CLK 1
jesd_import HD 1
jesd_import INTERP_CH 4
jesd_import INTERP_MAIN 8
jesd_import JESD_MODE 2
jesd_import JESDV 1
jesd_import K 32
jesd_import L 3
jesd_import LID 0
jesd_import LINK_DW 32
jesd_import M 6
jesd_import N 16
jesd_import NP 16
jesd_import PHADJ 0
jesd_import RES1 0
jesd_import RES2 0
jesd_import S 1
jesd_import SCR 1
jesd_import SUBCLASSV 1


----------------
 JESD mode 2
----------------
INTERP_CH: 4  INTERP_MAIN: 8  DSP_CLK_DIV: 32
JESD204BSettings(): 5a 05 00 82 03 1f 05 0f 2f 20 80 00 00 e6 
         DID:  90        BID:   5     ADJCNT:   0        LID:   0 
       PHADJ:   0     ADJDIR:   0          L:   3        SCR:   1 
           F:   4          K:  32          M:   6          N:  16 
          CS:   0         NP:  16  SUBCLASSV:   1          S:   1 
       JESDV:   1         CF:   0         HD:   1       RES1:   0 
        RES2:   0       FCHK: 230 
   [ LINK_DW:  32     FR_CLK:   1 ]

In [2]:
settings = Ad9174Settings(
    0, 4, 8,
    fchk_over_octets=True,
    SCR=1,
    DID=0x5A, 
    BID=0x05
)
print(settings)

----------------
 JESD mode 0
----------------
interp_ch: 4  interp_main: 8  dsp_clk_div: 32
JESD204BSettings(): 5a 05 00 80 03 1f 01 0f 2f 20 80 00 00 e0 
         DID:  90        BID:   5     ADJCNT:   0        LID:   0 
       PHADJ:   0     ADJDIR:   0          L:   1        SCR:   1 
           F:   4          K:  32          M:   2          N:  16 
          CS:   0         NP:  16  SUBCLASSV:   1          S:   1 
       JESDV:   1         CF:   0         HD:   1       RES1:   0 
        RES2:   0       FCHK: 224 
   [ LINK DW:  32   FR / CLK:   1 ]


In [3]:
r = conLitexServer('../build/csr.csv')

Connected to Port 1234
AD9174 + VC707 test 2020-04-24 00:11:06


In [4]:
ad = Ad9174Init(r, settings)

In [5]:
ad.init_hmc()

In [9]:
ad.print_fpga_clocks()

f_device = 160.000463 MHz  f_ref = 1.600005 MHz


In [7]:
ad.init_ad9174()
print('FPGA link ready / jsync: {:02b}'.format(r.regs.control_status.read()))
if ad.print_irq_flags(True):
    raise RuntimeError('init error')

AD917X_NVM_BLR_DONE: 1
PROD_ID: 0x9174
PROD_GRADE: 0  DEV_REVISION: 5
DLL locked: 1
SPI_PAGEINDX: 0b01000001
CAL_STAT: 1
SERDES PLL locked: 1
MODE_NOT_IN_TABLE: 0
SYNC_ROTATION_DONE 1
DYN_LINK_LATENCY  5 cycles
FPGA link ready / jsync: 11


In [13]:
ad.print_irq_flags(True)
print()

ad.print_ilas()

ad.print_lane_status()
print()

ad.test_stpl()


JESD settings, received on lane 0 vs (programmed):
450: 5a (5a)
451: 05 (05)
452: 00 (00)
453: 80 (80)
454: 03 (03)
455: 1f (1f)
456: 01 (01)
457: 0f (0f)
458: 2f (2f)
459: 20 (20)
45a: 80 (80)
45b: 00 (00)
45c: 00 (00)
45d: e0 (e0)
CHK: e0 (e0) 

Lane status:
      LANE_DESKEW: 00000001
    BAD_DISPARITY: 00000000
     NOT_IN_TABLE: 00000000
 UNEXPECTED_KCHAR: 00000000
    CODE_GRP_SYNC: 00000001
       FRAME_SYNC: 00000001
    GOOD_CHECKSUM: 00000001
   INIT_LANE_SYNC: 00000001
FIFO_STATUS_REG_0: 00000000
FIFO_STATUS_REG_1: 00000000
fpga j_sync errs: 4

STPL test:
converter: 0, tp: 597a, fail: 0
converter: 1, tp: d27a, fail: 0


In [10]:
ad.print_irq_flags(True)
ad.trigger_jref_sync()
print()
ad.print_irq_flags(True)

SYNC_ROTATION_DONE 1
DYN_LINK_LATENCY  5 cycles

JESD_IRQ_STATUSA: Code Group Sync. failed
JESD_IRQ_STATUSA: Frame Sync. failed
JESD_IRQ_STATUSA: ILAS checksum bad
IRQ_STATUS: JESD204x receiver not ready
IRQ_STATUS: SYSREF jitter too large


True

In [15]:
r.regs.phy0_transmitter_tp_on.write(0)

In [19]:
ad.print_phy_snapshot()

PHY_SNAPSHOT_DATA:
0: 0xab722b8f43, 0b1010101101110010001010111000111101000011
